- In this Notebook, I am trying to integrate the training data from the backscore and rest of the period from prj-prod-dataplatform.audit_balance.ml_model_run_details table.
- In this I will compare the training period with each month of test period.

**Steps to Follow**:

* Read the specific model data from prj-prod-dataplatform.audit_balance.ml_model_run_details table
* Expand the calcFeature column to extract all the features for the model
* Read the data from specific backscore table for the training data
* Identify the features and create a list
* Use transform_data function to create the same structure as ml_model_run_details table
* Insert the data to a similar training table - prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* Read the specific model data from prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* expand the training set from the calcFeature column
* Concatenate both the test and train datasets
* Calculate the PSI using the PSI function comparing it with the train set
* Insert the result to a PSI table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4

# **PSI - CSI Calculation**

## Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

## Function

In [2]:
# Method 1: Using regex to remove all "Calc_" occurrences
def clean_names_regex(name):
    return re.sub(r'_Calc_', '_', name)

# Method 2: Simple string replacement
def clean_names_replace(name):
    return name.replace('_Calc_', '_')

## expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_data = []
    
    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Handle None or NaN values
            if pd.isna(calc_features_str):
                calc_features_data.append({})
                continue
            
            # Convert to string if not already
            calc_features_str = str(calc_features_str)
            
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))
            
            # Ensure it's a dictionary, not a list
            if isinstance(features_dict, dict):
                calc_features_data.append(features_dict)
            elif isinstance(features_dict, list):
                # If it's a list, convert to dict with index keys or skip
                print(f"Warning: calcFeatures at index {idx} is a list, converting to dict")
                calc_features_data.append({'raw_list': features_dict})
            else:
                print(f"Warning: calcFeatures at index {idx} is neither dict nor list: {type(features_dict)}")
                calc_features_data.append({})
                
        except (json.JSONDecodeError, AttributeError, TypeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"  Value: {calc_features_str}")
            calc_features_data.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)
    
    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')
    
    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)
    
    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    return result_df

### dropping_duplicates

In [4]:
def dropping_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop duplicates based on digitalLoanAccountId, Data_selection, and modelVersionid,
    keeping the first occurrence based on appln_submit_datetime.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe

    Returns:
    --------
    pd.DataFrame with duplicates dropped
    """

    df = df.sort_values(
        ['digitalLoanAccountId', 'Data_selection', 'modelVersionId', 'appln_submit_datetime'],
        ascending=[True, True, True, True],
        na_position='last'
    )

    result = df.drop_duplicates(
        subset=['digitalLoanAccountId', 'Data_selection', 'modelVersionId'],
        keep='first'
    ).copy()

    return result

### PSI Pipeline Version 1

In [5]:
# import pandas as pd
# import numpy as np
# from datetime import datetime
# from typing import List, Dict, Optional, Tuple

# class PSICalculator:
#     """
#     A class to calculate Population Stability Index (PSI) with support for 
#     multiple segments and score columns.
#     """
    
#     def __init__(self, df: pd.DataFrame):
#         """
#         Initialize the PSI Calculator with a combined dataframe.
        
#         Parameters:
#         -----------
#         df : pd.DataFrame
#             Combined dataframe with train and test data
#         """
#         self.df = df.copy()
#         self._prepare_data()
    
#     def _prepare_data(self):
#         """Prepare and clean the data for PSI calculation."""
#         # Convert score columns to numeric
#         if 'score' in self.df.columns:
#             self.df['score'] = pd.to_numeric(self.df['score'], errors='coerce')
        
#         # Convert Application_month to datetime
#         if 'Application_month' in self.df.columns:
#             if self.df['Application_month'].dtype != 'datetime64[ns]':
#                 self.df['Application_month'] = pd.to_datetime(
#                     self.df['Application_month'] + '-01'
#                 )
        
#         # Standardize Data_selection column (handle case sensitivity)
#         if 'Data_selection' in self.df.columns:
#             self.df['Data_selection'] = self.df['Data_selection'].str.strip().str.title()
    
#     def _calculate_decile_bins(self, train_scores: pd.Series) -> np.ndarray:
#         """
#         Calculate decile bins from training data.
        
#         Parameters:
#         -----------
#         train_scores : pd.Series
#             Training set scores
        
#         Returns:
#         --------
#         np.ndarray
#             Array of bin edges for deciles
#         """
#         _, bins = pd.qcut(train_scores.dropna(), 10, labels=False, retbins=True)
#         return bins
    
#     def _get_distribution(
#         self, 
#         scores: pd.Series, 
#         bins: np.ndarray
#     ) -> pd.Series:
#         """
#         Get the decile distribution for a set of scores.
        
#         Parameters:
#         -----------
#         scores : pd.Series
#             Scores to bin
#         bins : np.ndarray
#             Decile bin edges
        
#         Returns:
#         --------
#         pd.Series
#             Distribution across deciles (0-9)
#         """
#         deciles = pd.cut(scores.dropna(), bins=bins, labels=False, include_lowest=True)
#         distribution = pd.Series(deciles).value_counts().sort_index() / len(scores)
        
#         # Align with all 10 deciles
#         all_bins = range(10)
#         distribution_aligned = distribution.reindex(all_bins, fill_value=0)
#         return distribution_aligned
    
#     def _calculate_psi_value(
#         self, 
#         current_dist: pd.Series, 
#         baseline_dist: pd.Series
#     ) -> float:
#         """
#         Calculate PSI value given two distributions.
        
#         Parameters:
#         -----------
#         current_dist : pd.Series
#             Current period distribution
#         baseline_dist : pd.Series
#             Baseline period distribution
        
#         Returns:
#         --------
#         float
#             PSI value
#         """
#         # Add small constant to avoid log(0)
#         epsilon = 1e-10
#         current_dist = current_dist + epsilon
#         baseline_dist = baseline_dist + epsilon
        
#         psi = (current_dist - baseline_dist) * np.log(current_dist / baseline_dist)
#         return psi.sum()
    
#     def calculate_psi_by_segment(
#         self,
#         score_column: str = 'score',
#         segment_columns: Optional[List[str]] = None,
#         date_column: str = 'Application_month',
#         data_selection_column: str = 'Data_selection',
#         account_id_column: str = 'digitalLoanAccountId'
#     ) -> pd.DataFrame:
#         """
#         Calculate PSI for each segment and month.
        
#         Parameters:
#         -----------
#         score_column : str
#             Name of the score column to analyze
#         segment_columns : Optional[List[str]]
#             List of columns to segment by. If None, calculates overall PSI only.
#             Example: ['modelVersionId', 'trenchCategory', 'loan_product_type']
#         date_column : str
#             Name of the application month column
#         data_selection_column : str
#             Name of the column distinguishing train/test data
#         account_id_column : str
#             Name of the account ID column for counting distinct accounts
        
#         Returns:
#         --------
#         pd.DataFrame
#             DataFrame with columns: Month, scorename, DateCategory, psivalues, 
#             account_count, segment_column, segment_value
#         """
        
#         # Validate inputs
#         if score_column not in self.df.columns:
#             raise ValueError(f"Score column '{score_column}' not found in dataframe")
        
#         if segment_columns is None:
#             segment_columns = []
        
#         # Check all required columns exist
#         required_cols = [date_column, data_selection_column, account_id_column, score_column]
#         missing_cols = [col for col in required_cols + segment_columns if col not in self.df.columns]
#         if missing_cols:
#             raise ValueError(f"Missing columns: {missing_cols}")
        
#         # Separate train and test data
#         train_df = self.df[self.df[data_selection_column].str.lower() == 'train'].copy()
#         test_df = self.df[self.df[data_selection_column].str.lower() == 'test'].copy()
        
#         if train_df.empty:
#             raise ValueError("No training data found")
#         if test_df.empty:
#             raise ValueError("No test data found")
        
#         # Calculate baseline decile bins from training data
#         train_bins = self._calculate_decile_bins(train_df[score_column])
#         baseline_dist = self._get_distribution(train_df[score_column], train_bins)
        
#         results = []
        
#         # Get last training month
#         last_train_month = train_df[date_column].max()
#         last_train_month_str = last_train_month.strftime('%Y-%m')
        
#         # Case 1: Overall PSI (no segments)
#         if not segment_columns:
#             # Add training set PSI
#             train_psi = self._calculate_psi_value(baseline_dist, baseline_dist)
#             results.append({
#                 'Month': last_train_month_str,
#                 'scorename': score_column,
#                 'DateCategory': 'Training',
#                 'psivalues': train_psi,
#                 'account_count': train_df[account_id_column].nunique(),
#                 'segment_column': None,
#                 'segment_value': None
#             })
            
#             # Calculate monthly PSI for test set
#             for month in sorted(test_df[date_column].unique()):
#                 month_str = month.strftime('%Y-%m')
#                 month_df = test_df[test_df[date_column] == month]
#                 if not month_df.empty:
#                     month_dist = self._get_distribution(month_df[score_column], train_bins)
#                     month_psi = self._calculate_psi_value(month_dist, baseline_dist)
#                     results.append({
#                         'Month': month_str,
#                         'scorename': score_column,
#                         'DateCategory': 'Monthly',
#                         'psivalues': month_psi,
#                         'account_count': month_df[account_id_column].nunique(),
#                         'segment_column': None,
#                         'segment_value': None
#                     })
        
#         # Case 2: Segmented PSI
#         else:
#             # For each segment column
#             for seg_col in segment_columns:
#                 # Training set PSI by segment
#                 for seg_val in train_df[seg_col].unique():
#                     if pd.isna(seg_val):
#                         continue
                    
#                     seg_train_df = train_df[train_df[seg_col] == seg_val]
#                     if not seg_train_df.empty:
#                         seg_baseline_dist = self._get_distribution(
#                             seg_train_df[score_column], 
#                             train_bins
#                         )
#                         seg_train_psi = self._calculate_psi_value(
#                             seg_baseline_dist, 
#                             seg_baseline_dist
#                         )
#                         results.append({
#                             'Month': last_train_month_str,
#                             'scorename': score_column,
#                             'DateCategory': 'Training',
#                             'psivalues': seg_train_psi,
#                             'account_count': seg_train_df[account_id_column].nunique(),
#                             'segment_column': seg_col,
#                             'segment_value': str(seg_val)
#                         })
                
#                 # Monthly PSI by segment for test set
#                 for month in sorted(test_df[date_column].unique()):
#                     month_str = month.strftime('%Y-%m')
#                     month_df = test_df[test_df[date_column] == month]
                    
#                     for seg_val in month_df[seg_col].unique():
#                         if pd.isna(seg_val):
#                             continue
                        
#                         seg_month_df = month_df[month_df[seg_col] == seg_val]
#                         if not seg_month_df.empty:
#                             seg_month_dist = self._get_distribution(
#                                 seg_month_df[score_column], 
#                                 train_bins
#                             )
#                             seg_month_psi = self._calculate_psi_value(
#                                 seg_month_dist, 
#                                 baseline_dist
#                             )
#                             results.append({
#                                 'Month': month_str,
#                                 'scorename': score_column,
#                                 'DateCategory': 'Monthly',
#                                 'psivalues': seg_month_psi,
#                                 'account_count': seg_month_df[account_id_column].nunique(),
#                                 'segment_column': seg_col,
#                                 'segment_value': str(seg_val)
#                             })
        
#         return pd.DataFrame(results)


# # Usage Examples:
# # ===============

# # Example 1: Overall PSI (no segments)
# # psi_calc = PSICalculator(df_

### PSI Pipeline Version 2

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import product, combinations

def calculate_psi_with_segments(
    df,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=None,
    num_bins=10,
    train_label='Train',
    test_label='Test'
):
    """
    Calculate Population Stability Index (PSI) with waterfall segmentation support.
    Creates waterfall combinations: first segment alone, then segment1+segment2, 
    then segment1+segment2+segment3, etc.
    Each segment uses its own training distribution.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Combined dataframe with train and test data
    score_column : str
        Name of the score column (default: 'score')
    date_column : str
        Name of the date column (default: 'Application_month')
    data_selection_column : str
        Name of the column indicating train/test split (default: 'Data_selection')
    segment_columns : list
        List of column names to segment by in waterfall order.
        Example: ['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
    num_bins : int
        Number of bins/deciles for PSI calculation (default: 10)
    train_label : str
        Value indicating training data (default: 'Train')
    test_label : str
        Value indicating test data (default: 'Test')
    
    Returns:
    --------
    pd.DataFrame
        Output dataframe with columns: Month, scorename, DateCategory, psivalues, 
        account_count, modelVersionId, trenchCategory, new_loan_type, 
        loan_product_type, osType
    """
    
    # Create a copy to avoid modifying original data
    df = df.copy()
    
    # Convert score to numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    
    # Convert Application_month to datetime if needed
    if df[date_column].dtype != 'datetime64[ns]':
        df[date_column] = pd.to_datetime(df[date_column].astype(str) + '-01', errors='coerce')
    
    # Separate train and test data
    train_df = df[df[data_selection_column] == train_label].copy()
    test_df = df[df[data_selection_column] == test_label].copy()
    
    # Handle missing values
    train_df = train_df.dropna(subset=[score_column])
    test_df = test_df.dropna(subset=[score_column])
    
    if train_df.empty or test_df.empty:
        raise ValueError("Train or Test dataframe is empty after removing NaN values")
    
    # If no segment columns specified, calculate overall PSI
    if segment_columns is None or len(segment_columns) == 0:
        segment_columns = []
    
    # Function to calculate PSI with predefined bins
    def calculate_psi_with_bins(data_scores, train_distribution, decile_bins):
        """
        Calculate PSI using pre-defined decile bins from training data.
        """
        if len(data_scores) == 0:
            return np.nan
        
        # Bin the scores using the training bins
        data_binned = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
        
        # Get distribution of current data
        current_dist = pd.Series(data_binned).value_counts().sort_index() / len(data_scores)
        
        # Align with training distribution (ensure all bins are represented)
        all_bins = range(num_bins)
        current_dist_aligned = current_dist.reindex(all_bins, fill_value=1e-10)
        train_dist_aligned = train_distribution.reindex(all_bins, fill_value=1e-10)
        
        # Calculate PSI
        psi_values = (current_dist_aligned - train_dist_aligned) * np.log(current_dist_aligned / train_dist_aligned)
        
        return psi_values.sum()
    
    # Get the last month of the training set
    last_train_month = train_df[date_column].max()
    last_train_month_str = last_train_month.strftime('%Y-%m')
    
    # Initialize results list
    psi_results = []
    
    # Define all possible segment columns (for creating NA columns)
    all_possible_segments = ['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
    
    # If no segments, calculate overall PSI
    if len(segment_columns) == 0:
        # Calculate training deciles and distribution
        train_score_binned = pd.qcut(train_df[score_column], num_bins, labels=False, duplicates='drop')
        train_distribution = train_score_binned.value_counts().sort_index() / len(train_df[score_column])
        _, train_decile_bins = pd.qcut(train_df[score_column], num_bins, retbins=True, duplicates='drop')
        
        # Training set PSI
        train_psi = calculate_psi_with_bins(train_df[score_column], train_distribution, train_decile_bins)
        result_row = {
            'Month': last_train_month_str,
            'scorename': score_column,
            'DateCategory': 'Training',
            'psivalues': train_psi,
            'account_count': train_df['digitalLoanAccountId'].nunique(),
        }
        # Add all segment columns as NA
        for seg_col in all_possible_segments:
            result_row[seg_col] = 'OVER ALL'
        psi_results.append(result_row)
        
        # Monthly PSI for test set
        for month in sorted(test_df[date_column].unique()):
            month_str = month.strftime('%Y-%m')
            month_df = test_df[test_df[date_column] == month]
            
            if not month_df.empty:
                month_psi = calculate_psi_with_bins(month_df[score_column], train_distribution, train_decile_bins)
                result_row = {
                    'Month': month_str,
                    'scorename': score_column,
                    'DateCategory': 'Monthly',
                    'psivalues': month_psi,
                    'account_count': month_df['digitalLoanAccountId'].nunique(),
                }
                for seg_col in all_possible_segments:
                    result_row[seg_col] = 'OVER ALL'
                psi_results.append(result_row)
    
    else:
        # Create waterfall combinations
        # For waterfall: [col1], [col1, col2], [col1, col2, col3], etc.
        waterfall_combinations = []
        for i in range(1, len(segment_columns) + 1):
            waterfall_combinations.append(segment_columns[:i])
        
        # Process each waterfall level
        for current_segments in waterfall_combinations:
            
            # Get unique values for each segment column in current waterfall level
            segment_combinations = []
            for seg_col in current_segments:
                unique_vals = df[seg_col].dropna().unique()
                segment_combinations.append([(seg_col, val) for val in unique_vals])
            
            # Generate all combinations for this waterfall level
            for seg_combo in product(*segment_combinations):
                # Filter data for this segment combination
                seg_filter = df.copy()
                
                for seg_col, seg_val in seg_combo:
                    seg_filter = seg_filter[seg_filter[seg_col] == seg_val]
                
                if seg_filter.empty:
                    continue
                
                # Split by train/test for this segment
                seg_train = seg_filter[seg_filter[data_selection_column] == train_label].copy()
                seg_test = seg_filter[seg_filter[data_selection_column] == test_label].copy()
                
                # Skip if segment has no data
                if seg_train.empty or seg_test.empty:
                    continue
                
                # Remove NaN values
                seg_train = seg_train.dropna(subset=[score_column])
                seg_test = seg_test.dropna(subset=[score_column])
                
                if seg_train.empty or seg_test.empty:
                    continue
                
                # Calculate training distribution SPECIFIC to this segment
                seg_train_binned = pd.qcut(
                    seg_train[score_column],
                    num_bins,
                    labels=False,
                    duplicates='drop'
                )
                seg_train_dist = seg_train_binned.value_counts().sort_index() / len(seg_train[score_column])
                
                # Get decile bins specific to this segment's training data
                _, seg_train_decile_bins = pd.qcut(
                    seg_train[score_column],
                    num_bins,
                    retbins=True,
                    duplicates='drop'
                )
                
                # Training set PSI for this segment
                seg_train_psi = calculate_psi_with_bins(seg_train[score_column], seg_train_dist, seg_train_decile_bins)
                
                # Create result row for training
                result_row = {
                    'Month': last_train_month_str,
                    'scorename': score_column,
                    'DateCategory': 'Training',
                    'psivalues': seg_train_psi,
                    'account_count': seg_train['digitalLoanAccountId'].nunique(),
                }
                
                # Add segment values
                for seg_col, seg_val in seg_combo:
                    result_row[seg_col] = seg_val
                
                # Add NA for segments not in this waterfall level
                for seg_col in all_possible_segments:
                    if seg_col not in [col for col, _ in seg_combo]:
                        result_row[seg_col] = 'OVER ALL'
                
                psi_results.append(result_row)
                
                # Monthly PSI for test set (this segment)
                for month in sorted(seg_test[date_column].unique()):
                    month_str = month.strftime('%Y-%m')
                    month_df = seg_test[seg_test[date_column] == month]
                    
                    if not month_df.empty:
                        month_psi = calculate_psi_with_bins(month_df[score_column], seg_train_dist, seg_train_decile_bins)
                        
                        result_row = {
                            'Month': month_str,
                            'scorename': score_column,
                            'DateCategory': 'Monthly',
                            'psivalues': month_psi,
                            'account_count': month_df['digitalLoanAccountId'].nunique(),
                        }
                        
                        # Add segment values
                        for seg_col, seg_val in seg_combo:
                            result_row[seg_col] = seg_val
                        
                        # Add NA for segments not in this waterfall level
                        for seg_col in all_possible_segments:
                            if seg_col not in [col for col, _ in seg_combo]:
                                result_row[seg_col] = 'OVER ALL'
                        
                        psi_results.append(result_row)
    
    # Create output dataframe
    output_df = pd.DataFrame(psi_results)
    
    # Reorder columns
    column_order = ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count'] + all_possible_segments
    output_df = output_df[[col for col in column_order if col in output_df.columns]]
    
    return output_df


# USAGE EXAMPLES:
# ===============

# Example 1: Waterfall segmentation with all levels
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
# )

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

# print(f"Output shape: {score_output_df.shape}")
# print(f"\nOutput columns: {score_output_df.columns.tolist()}")
# print(f"\nFirst few rows:")
# print(score_output_df.head(20))
# print(f"\nData types:")
# print(score_output_df.dtypes)

# Config query

In [7]:
sq = """ WITH base AS (
  -- First part with actual trenchCategory
  SELECT 
    modelDisplayName, 
    modelVersionId, 
    CASE 
      WHEN trenchCategory IS NULL THEN 'ALL'
      WHEN trenchCategory = '' THEN 'ALL'
      ELSE trenchCategory 
    END AS trenchCategory,
(CASE 
      WHEN modelDisplayName LIKE '%sil%' THEN 'SIL'
      WHEN modelDisplayName LIKE '%cash%' THEN 'CASH'
      ELSE 'ALL' 
    END) AS product_category,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`

  
  UNION ALL
  
  -- Second part with 'ALL' trenchCategory
  SELECT 
    modelDisplayName, 
    modelVersionId, 
    'ALL' AS trenchCategory,
    (CASE 
      WHEN modelDisplayName LIKE '%sil%' THEN 'SIL'
      WHEN modelDisplayName LIKE '%cash%' THEN 'CASH'
      ELSE 'ALL' 
    END) AS product_category,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE trenchCategory IS NOT NULL

)
SELECT distinct
  modelDisplayName, 
  modelVersionId, 
  trenchCategory, 
  product_category,
FROM base
ORDER BY 4, 1, 2, 3;"""

configdf = client.query(sq).to_dataframe()
configdf

,modelDisplayName,modelVersionId,trenchCategory,product_category
0,alpha_income_model,v1,ALL,ALL
1,beta_income_model,v1,ALL,ALL
2,beta_income_model,v1,Trench 1,ALL
3,beta_income_model,v1,Trench 2,ALL
4,beta_income_model,v1,Trench 3,ALL
5,alpha_stack_model_cash,v1,ALL,CASH
6,alpha_stack_model_cash,v1,Trench 1,CASH
7,alpha_stack_model_cash,v1,Trench 2,CASH
8,alpha_stack_model_cash,v1,Trench 3,CASH
9,alpha_stack_model_cash,v1.1,ALL,CASH


In [8]:
# configdf.to_csv('configdf.csv', index = False)

## SIL

### Queries

### cic_model_sil

### Test

In [9]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details

sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
        case when modelDisplayName = 'Alpha - CIC-SIL-Model' then 'cic_model_sil' else modelDisplayName end as modelDisplayName    
    ,modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
base as
(SELECT distinct

--Alpha_cic_sil_score
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'cic_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
)
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

The shape of the dataframe is: (114178, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1744871,2e1eb846-4e02-405f-aa08-01770578e647,0.14526991814317072,2025-06-15 10:15:12.304204,2025-06-15 10:15:12.310663,cic_model_sil,v1,SIL-Instore,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1749945600000,""cic_Personal_Loans_...",Test,2025-06-15 18:15:04,NaT,2025-06
1,1955033,ee5b45a8-9fc1-4649-bff3-72127d8ebf69,0.09946015604218136,2025-06-07 06:35:50.326717,2025-06-07 06:35:50.332658,cic_model_sil,v1,SIL-Instore,F,Appliance,ios,cic_model_sil,SIL,ALL,"{""run_date"":1749254400000,""cic_Personal_Loans_...",Test,2025-06-07 14:35:39,NaT,2025-06
2,2288963,3d00407c-5ef0-4d5d-adc4-6c84f53705f7,0.1855724465004623,2025-10-01 11:15:24.595429,2025-10-01 11:15:24.601903,cic_model_sil,v1,SIL-Instore,F,Mall,ios,cic_model_sil,SIL,ALL,"{""run_date"":1759276800000,""cic_Personal_Loans_...",Test,2025-10-01 19:15:12,2025-10-01 19:18:54,2025-10
3,2484341,4292d504-33f6-4ba4-af58-7e0b8124a7b0,0.1008692326571366,2025-07-04 09:46:17.621366,2025-07-04 09:46:17.627639,cic_model_sil,v1,SIL-Instore,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1751587200000,""cic_Personal_Loans_...",Test,2025-07-04 17:46:08,2025-07-04 18:05:30,2025-07
4,2498960,0b5dd0ef-9439-4d5c-be4e-0352d04dea74,0.1802557442554918,2025-09-18 11:41:04.502090,2025-09-18 11:41:04.508570,cic_model_sil,v1,SIL-Instore,F,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1758153600000,""cic_Personal_Loans_...",Test,2025-09-18 19:40:53,NaT,2025-09


In [10]:
df1 = dfd.copy()

### Train

In [11]:
sq = """WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    
    case when modelDisplayName = 'Alpha - CIC-SIL-Model' then 'cic_model_sil' else modelDisplayName end as modelDisplayName
    
    ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
base as 
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (286189, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1069495,8d72ddd0-d1a5-49f8-925d-a33548e1deac,0.152116,2025-12-19T13:11:24.108742,2025-12-19T13:11:24.108742,cic_model_sil,v1,SIL-Instore,M,Mobile,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2024-09-27 17:51:49,NaT,2024-09
1,1364254,42c29602-f870-4ccd-b9b2-b39c7eeab813,0.470879,2025-12-19T13:21:57.047829,2025-12-19T13:21:57.047829,cic_model_sil,v2,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,Trench 2,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Train,2024-12-09 13:41:46,NaT,2024-12
2,1879530,6676fbdf-40a8-4d94-9975-39f5672df0b6,0.222979,2025-12-19T13:22:01.548095,2025-12-19T13:22:01.548095,cic_model_sil,v2,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,Trench 3,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Train,2024-10-06 16:49:56,2024-10-06 17:13:56,2024-10
3,1953449,7dd2ae5b-3347-4f6e-91d4-3f9f34580826,0.125133,2025-12-19T13:11:27.763233,2025-12-19T13:11:27.763233,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_days_since_last_inquiry"": 1211.0, ""cic_z...",Train,2023-03-21 16:44:06,NaT,2023-03
4,2011484,f3901959-4a11-43bf-9537-3e5f6c098d91,0.201671,2025-12-19T13:11:34.743234,2025-12-19T13:11:34.743234,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_days_since_last_inquiry"": 357.0, ""cic_cn...",Train,2023-04-24 12:07:36,2023-04-24 12:13:14,2023-04


In [12]:
df2 = dfd.copy()

In [13]:
df2[['Application_month']][(df2['modelVersionId'] =='v1')].max()

Application_month    2025-01
dtype: object

In [14]:
df2[df2['Application_month'] == '2025-11']

,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month


In [15]:
dd.query("""select Application_month, count(distinct digitalLoanAccountId)cnt from df2 
         where modelVersionId ='v1'
         group by 1 order by 1 desc;""").to_df()

,Application_month,cnt
0,2025-01,16318
1,2024-12,34696
2,2024-11,17931
3,2024-10,16817
4,2024-09,17502
5,2024-08,17808
6,2024-07,13346
7,2024-06,13472
8,2024-05,8033
9,2024-04,5851


In [16]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (400367, 19)


In [17]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (400367, 19)
The shape of the dataframe after dropping duplicates is: (400367, 19)


In [18]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [19]:
df_combined['Application_month'][df_combined['Data_selection'] == 'Train'].max()

'2025-01'

### PSI calculation

In [20]:
# USAGE EXAMPLES:
# ===============

# Example 1: Calculate overall PSI (no segmentation)
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=None
# )

# Example 2: Calculate PSI with single segmentation (modelVersionId)
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId']
# )

# Example 3: Calculate PSI with multiple segmentations (all combinations)
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
# )

# Example 4: With a different score column name
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='risk_score',  # Different score column
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId', 'trenchCategory']
# )

# Example 1: Waterfall segmentation with all levels
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (366, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-01     score     Training   0.000000         184484             v1   
1   2025-03     score      Monthly   0.134249            180             v1   
2   2025-04     score      Monthly   0.110920            541             v1   
3   2025-05     score      Monthly   0.108170            858             v1   
4   2025-06     score      Monthly   0.120669           9934             v1   
5   2025-07     score      Monthly   0.127452          14043             v1   
6   2025-08     score      Monthly   0.134328          15085             v1   
7   2025-09     score      Monthly   0.121338          13880             v1   
8   2025-10     score      Monthly   0.129319          16424             v1   

In [21]:
# # Initialize calculator
# psi_calc = PSICalculator(df_combined)
# results_overall = psi_calc.calculate_psi_by_segment(score_column='score')

In [22]:
# # 2. PSI by individual segment
# results_modelversion = psi_calc.calculate_psi_by_segment(
#     score_column='score',
#     segment_columns=['modelVersionId']
# )

In [23]:
# results_modelversion

In [24]:
# results_multi = psi_calc.calculate_psi_by_segment(
#     score_column='score',
#     segment_columns=['modelVersionId', 'trenchCategory', 'loan_product_type', 'osType']
# )

In [25]:
score_output_df

score_output_df['Model_Name'] = 'cic_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'Alpha_cic_sil_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-01,Alpha_cic_sil_score,Training,0.000000,184484,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_sil
1,2025-03,Alpha_cic_sil_score,Monthly,0.134249,180,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_sil
2,2025-04,Alpha_cic_sil_score,Monthly,0.110920,541,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_sil
3,2025-05,Alpha_cic_sil_score,Monthly,0.108170,858,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_sil
4,2025-06,Alpha_cic_sil_score,Monthly,0.120669,9934,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
361,2025-11,Alpha_cic_sil_score,Monthly,13.939886,6,v2,Trench 2,SIL Competitor,Mall,android,cic_model_sil
362,2025-12,Alpha_cic_sil_score,Monthly,5.453970,50,v2,Trench 2,SIL Competitor,Mall,android,cic_model_sil
363,2025-01,Alpha_cic_sil_score,Training,0.000000,1,v2,Trench 2,SIL Competitor,Mall,ios,cic_model_sil
364,2025-11,Alpha_cic_sil_score,Monthly,0.000000,1,v2,Trench 2,SIL Competitor,Mall,ios,cic_model_sil


In [26]:
# # 4. With custom column names
# results_custom = psi_calc.calculate_psi_by_segment(
#     score_column='score',
#     segment_columns=['modelVersionId', 'new_loan_type'],
#     account_id_column='digitalLoanAccountId',
#     data_selection_column='Data_selection'
# )

In [27]:
# copy the data to psi_results
psi_results = score_output_df.copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1822399a-58ab-4dfc-b150-84ef4884cd8d>

### Alpha Sil Stack Model

### Test

In [28]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details
# Sil_Alpha_Stack_score
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
        case when modelDisplayName = 'Alpha - StackingModel' then 'alpha_stack_model_sil' else modelDisplayName end as modelDisplayName    
    ,modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
base as
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'alpha_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
)
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

The shape of the dataframe is: (114182, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,2186836,89d2b064-fef8-42ca-9db8-289a4328c4d5,0.46711628730392696,2025-12-17 05:19:17.502820,2025-12-17 05:19:17.674271,alpha_stack_model_sil,v2,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,Trench 3,"{""sb_demo_score"": 0.4005454828, ""apps_score"": ...",Test,2025-12-17 13:19:05,2025-12-17 13:34:21,2025-12
1,2592766,a18feda1-8fdd-49f9-a4e7-a0d971c09ea9,0.03503865209582487,2025-07-23 10:35:39.605342,2025-07-23 10:35:39.611684,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.0918237408, ""s_cic_score"":...",Test,2025-07-23 18:35:29,NaT,2025-07
2,2598340,b6cf4154-bf7b-4b0d-83cf-f32460884004,0.1638506399821919,2025-09-21 05:50:43.906149,2025-09-21 05:50:43.911879,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.1528674406, ""s_cic_score"":...",Test,2025-09-21 13:50:22,NaT,2025-09
3,2737598,61216c3c-ef2c-443b-8809-a2616340658c,0.15122853593455846,2025-11-30 05:05:19.592875,2025-11-30 05:05:19.728662,alpha_stack_model_sil,v2,SIL-Instore,M,Appliance,android,alpha_stack_model_sil,SIL,Trench 2,"{""sb_demo_score"": 0.3177949106, ""apps_score"": ...",Test,2025-11-30 13:05:07,2025-11-30 13:07:38,2025-11
4,2793825,b2a5a5e3-a848-4a30-94a2-020727f5db6c,0.18752811038830144,2025-11-21 06:36:33.425720,2025-11-21 06:36:33.431485,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.1599150797, ""s_cic_score"":...",Test,2025-11-21 14:36:22,2025-11-21 14:39:27,2025-11


In [29]:
df1 = dfd.copy()

### Train

In [30]:
sq = """WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    
    case when modelDisplayName = 'Alpha - StackingModel' then 'alpha_stack_model_sil' else modelDisplayName end as modelDisplayName 
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
base as 
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'alpha_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (300166, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1404246,4c16f12c-086c-47a6-84be-94d9f2d877f4,0.541360,2025-12-19T13:22:23.566469,2025-12-19T13:22:23.566469,alpha_stack_model_sil,v2,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,Trench 2,"{""sb_demo_score"": 0.384161150933734, ""apps_sco...",Train,2024-12-24 09:27:30,NaT,2024-12
1,1862619,70c70eed-3c9e-44e8-a431-b97f261c0090,0.019650,2025-12-19T13:12:01.589390,2025-12-19T13:12:01.589390,alpha_stack_model_sil,v1,SIL ZERO,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.042025782740490354, ""s_cic...",Train,2024-10-21 17:25:57,2024-10-21 17:30:45,2024-10
2,1916099,aada62f9-be68-4a5d-bd47-d0aa04853935,0.218282,2025-12-19T13:12:05.297708,2025-12-19T13:12:05.297708,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.09307789932750118, ""s_cic_...",Train,2023-10-31 18:08:46,2023-10-31 18:36:50,2023-10
3,1925647,4b524327-1f43-4b6c-98f6-135941e1631e,0.440634,2025-12-19T13:22:23.481754,2025-12-19T13:22:23.481754,alpha_stack_model_sil,v2,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,Trench 2,"{""sb_demo_score"": 0.45265519757887607, ""apps_s...",Train,2024-12-22 13:56:01,2024-12-22 14:09:58,2024-12
4,2008962,dc85f12a-3c02-4bd8-9d9c-4739fe1b4185,0.093095,2025-12-19T13:11:55.136934,2025-12-19T13:11:55.136934,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.16114475726174338, ""s_cic_...",Train,2023-04-22 16:44:33,NaT,2023-04


In [31]:
df2 = dfd.copy()

In [32]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (414348, 19)


In [33]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (414348, 19)
The shape of the dataframe after dropping duplicates is: (414348, 19)


In [34]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [35]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (369, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-02     score     Training   0.000000         115682             v2   
1   2025-11     score      Monthly   0.574844            944             v2   
2   2025-12     score      Monthly   0.044039           9807             v2   
3   2025-02     score     Training   0.000000         184484             v1   
4   2025-03     score      Monthly  12.685199            180             v1   
5   2025-04     score      Monthly   0.176158            541             v1   
6   2025-05     score      Monthly   0.041444            858             v1   
7   2025-06     score      Monthly   0.100930           9934             v1   
8   2025-07     score      Monthly   0.107424          14043             v1   

In [36]:
score_output_df

score_output_df['Model_Name'] = 'alpha_stack_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'Sil_Alpha_Stack_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-02,Sil_Alpha_Stack_score,Training,0.000000,115682,v2,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_sil
1,2025-11,Sil_Alpha_Stack_score,Monthly,0.574844,944,v2,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_sil
2,2025-12,Sil_Alpha_Stack_score,Monthly,0.044039,9807,v2,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_sil
3,2025-02,Sil_Alpha_Stack_score,Training,0.000000,184484,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_sil
4,2025-03,Sil_Alpha_Stack_score,Monthly,12.685199,180,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
364,2025-08,Sil_Alpha_Stack_score,Monthly,0.175997,234,v1,ALL,SIL Competitor,Mall,ios,alpha_stack_model_sil
365,2025-09,Sil_Alpha_Stack_score,Monthly,0.153353,215,v1,ALL,SIL Competitor,Mall,ios,alpha_stack_model_sil
366,2025-10,Sil_Alpha_Stack_score,Monthly,0.141827,295,v1,ALL,SIL Competitor,Mall,ios,alpha_stack_model_sil
367,2025-11,Sil_Alpha_Stack_score,Monthly,0.187239,372,v1,ALL,SIL Competitor,Mall,ios,alpha_stack_model_sil


In [37]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2340720b-87c8-408c-9bb9-a633c25614d7>

### Beta Sil App Score

### Test

In [38]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
    
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'apps_score_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_app_score
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base 
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1955152,3405491a-a4f0-4a3a-9bd1-0efd0d6a2bce,"{'cb_score': 0.3544518316002896, 'dl_score': 0...",2025-11-18 11:09:45.331847,2025-11-18 11:09:45.794058,apps_score_model_sil,v1,SIL-Instore,M,Mall,android,apps_score_model_sil,SIL,ALL,Test,0.355791,"{""app_cnt_rated_for_3plus_ever"":38,""app_cnt_ed...",2025-11-18 19:09:42,NaT,2025-11
1,2124821,4cf22e60-5ef8-405a-9d85-58671192de9c,"{'cb_score': 0.4596568861189064, 'dl_score': 0...",2025-06-17 14:53:53.476152,2025-06-17 14:53:53.895764,apps_score_model_sil,v1,SIL-Instore,F,Mall,android,apps_score_model_sil,SIL,ALL,Test,0.541465,"{""app_cnt_rated_for_3plus_ever"":12,""app_cnt_ed...",2025-06-15 14:55:19,NaT,2025-06
2,2173429,9f21ab7f-947a-4b0f-b347-98a3f1c6045b,"{'cb_score': 0.5303502854162367, 'dl_score': 0...",2025-11-20 08:07:52.753549,2025-11-20 08:07:53.203288,apps_score_model_sil,v1,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,ALL,Test,0.574448,"{""app_cnt_rated_for_3plus_ever"":34,""app_cnt_ed...",2025-11-20 16:07:34,NaT,2025-11
3,2177753,e5360fd6-acd3-43a5-b746-03ef7444246a,"{'cb_score': 0.6370279671114918, 'dl_score': 0...",2025-10-12 02:58:52.043726,2025-10-12 02:58:52.559078,apps_score_model_sil,v1,SIL Competitor,F,Mall,android,apps_score_model_sil,SIL,ALL,Test,0.675122,"{""app_cnt_rated_for_3plus_ever"":8,""app_cnt_edu...",2025-10-12 10:58:48,NaT,2025-10
4,2248419,6a1f2aea-c85c-49e1-9ade-08d60bc3ed68,"{'cb_score': 0.39649294816369807, 'dl_score': ...",2025-08-10 05:14:57.880670,2025-08-10 05:14:58.317941,apps_score_model_sil,v1,SIL-Instore,F,Mall,android,apps_score_model_sil,SIL,ALL,Test,0.477360,"{""app_cnt_rated_for_3plus_ever"":54,""app_cnt_ed...",2025-08-10 13:14:55,NaT,2025-08


In [39]:
df1 = dfd.copy()

### Train

In [40]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil' else modelDisplayName end as modelDisplayName 
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'apps_score_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base 
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1055817,87638f02-a75b-46c1-a287-15c391582302,0.411336,2025-12-19T13:24:14.206487,2025-12-19T13:24:14.206487,apps_score_model_sil,v2,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,Trench 3,Train,0.411336,"{""app_cnt_productivity_ever"": 17.0, ""app_cnt_r...",2025-07-01 14:18:42,2025-07-01 14:22:05,2025-07
1,1067101,3a835f00-96b4-437e-a94e-779480f0097c,0.482263,2025-12-19T13:24:04.914778,2025-12-19T13:24:04.914778,apps_score_model_sil,v2,SIL-Instore,F,Mall,android,apps_score_model_sil,SIL,Trench 2,Train,0.482263,"{""app_cnt_payday_ever"": 7, ""app_cnt_finance_ev...",2025-02-03 14:27:28,NaT,2025-02
2,1236079,4c0b90e4-383c-4fb9-83fd-106b3972ce5c,0.341078,2025-12-19T13:24:04.895779,2025-12-19T13:24:04.895779,apps_score_model_sil,v2,SIL Competitor,F,Appliance,android,apps_score_model_sil,SIL,Trench 2,Train,0.341078,"{""app_cnt_payday_ever"": 6, ""app_cnt_finance_ev...",2025-10-15 14:14:53,NaT,2025-10
3,1245314,273678a9-bbed-4b29-96ff-eee6e22cc05e,0.394100,2025-12-19T13:24:12.058915,2025-12-19T13:24:12.058915,apps_score_model_sil,v2,SIL ZERO,M,Appliance,android,apps_score_model_sil,SIL,Trench 3,Train,0.394100,"{""app_cnt_productivity_ever"": 8.0, ""app_cnt_ra...",2025-04-07 14:19:34,NaT,2025-04
4,1364254,42c29602-f870-4ccd-b9b2-b39c7eeab813,0.414331,2025-12-19T13:24:04.722380,2025-12-19T13:24:04.722380,apps_score_model_sil,v2,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,Trench 2,Train,0.414331,"{""app_cnt_payday_ever"": 4, ""app_cnt_finance_ev...",2024-12-09 13:41:46,NaT,2024-12


In [41]:
df2 = dfd.copy()

In [42]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (599248, 20)


In [43]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (599248, 20)
The shape of the dataframe after dropping duplicates is: (599248, 20)


In [44]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [45]:
dd.query("""select modelVersionId, Data_selection, count(distinct digitalLoanAccountId) cnt from df_combined group by 1,2;""").to_df()

,modelVersionId,Data_selection,cnt
0,v2,Train,328451
1,v2,Test,13901
2,v1,Test,140863
3,v1,Train,116033


### PSI calculation

In [46]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (55, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-11     score     Training   0.000000         116033             v1   
1   2025-03     score      Monthly   0.075090            373             v1   
2   2025-04     score      Monthly   0.079331            914             v1   
3   2025-05     score      Monthly   0.061396           1389             v1   
4   2025-06     score      Monthly   0.039468          14507             v1   
5   2025-07     score      Monthly   0.041627          20089             v1   
6   2025-08     score      Monthly   0.033829          20563             v1   
7   2025-09     score      Monthly   0.044746          19126             v1   
8   2025-10     score      Monthly   0.051219          22029             v1   


In [47]:
score_output_df

score_output_df['Model_Name'] = 'apps_score_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'sil_beta_app_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-11,sil_beta_app_score,Training,0.000000,116033,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
1,2025-03,sil_beta_app_score,Monthly,0.075090,373,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
2,2025-04,sil_beta_app_score,Monthly,0.079331,914,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
3,2025-05,sil_beta_app_score,Monthly,0.061396,1389,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
4,2025-06,sil_beta_app_score,Monthly,0.039468,14507,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
5,2025-07,sil_beta_app_score,Monthly,0.041627,20089,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
6,2025-08,sil_beta_app_score,Monthly,0.033829,20563,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
7,2025-09,sil_beta_app_score,Monthly,0.044746,19126,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
8,2025-10,sil_beta_app_score,Monthly,0.051219,22029,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil
9,2025-11,sil_beta_app_score,Monthly,0.055074,25032,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_model_sil


In [48]:
score_output_df.modelVersionId.unique()

array(['v1'], dtype=object)

In [49]:
psi_results = score_output_df.copy()

table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=66aca3b1-fad2-4e78-83a3-f64aaecd13c9>

## Beta SIL Demo Score

### Test

In [50]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_demo_score
  prediction as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base where lower(new_loan_type) like '%sil%'
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1030731,be49a865-451d-480b-b6d1-c01c9d139473,0.0464854196,2025-08-21 09:08:10.177283,2025-08-21 09:08:10.239909,beta_demo_model_sil,v1,SIL-Instore,F,Mall,android,beta_demo_model_sil,SIL,ALL,Test,0.0464854196,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-08-21 17:08:07,2025-08-21 17:31:45,2025-08
1,1396915,c53cd8e5-9266-4a41-8456-0da6bf17d9c3,0.1511337633,2025-08-02 11:58:09.484811,2025-08-02 11:58:09.546475,beta_demo_model_sil,v1,SIL-Instore,F,Mall,ios,beta_demo_model_sil,SIL,ALL,Test,0.1511337633,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-08-02 19:58:07,NaT,2025-08
2,1770925,6fcf8ec3-d877-4fd2-af05-379ec634f966,0.11065524,2025-11-25 10:40:25.984727,2025-11-25 10:40:26.053777,beta_demo_model_sil,v1,SIL-Instore,F,Mall,android,beta_demo_model_sil,SIL,ALL,Test,0.11065524,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-11-25 18:40:22,2025-11-25 18:54:58,2025-11
3,2136564,9c4d2153-956d-4796-ac85-9070ed1f5c66,0.0850273867,2025-06-29 06:27:30.415153,2025-06-29 06:27:30.477980,beta_demo_model_sil,v1,SIL Competitor,F,Appliance,android,beta_demo_model_sil,SIL,ALL,Test,0.0850273867,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-06-29 14:27:27,NaT,2025-06
4,2145919,f843de3c-ab0d-4869-914e-5268bce8db31,0.15917279,2025-10-28 03:10:12.088136,2025-10-28 03:10:12.151315,beta_demo_model_sil,v1,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,ALL,Test,0.15917279,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-10-28 11:10:09,NaT,2025-10


In [51]:
df1 = dfd.copy()

### Train

In [52]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName    
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  prediction as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base where lower(new_loan_type) like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1101533,415635dc-d86d-4cb6-b100-417d540fcf38,0.149562,2025-12-19T13:13:43.748145,2025-12-19T13:13:43.748145,beta_demo_model_sil,v1,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.149562,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2024-08-26 17:41:44,NaT,2024-08
1,1171326,f253f60b-8051-465b-930c-844b5e936be2,0.167007,2025-12-19T13:13:00.786644,2025-12-19T13:13:00.786644,beta_demo_model_sil,v1,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.167007,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2023-09-17 10:23:54,2023-09-17 10:37:28,2023-09
2,1250842,1704b9a3-6b2c-4207-882e-df299bf26f6a,0.470330,2025-12-19T13:24:58.133141,2025-12-19T13:24:58.133141,beta_demo_model_sil,v2,SIL ZERO,M,Mobile,android,beta_demo_model_sil,SIL,Trench 2,Train,0.470330,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",2024-12-23 17:44:29,2024-12-26 16:46:28,2024-12
3,1328007,7144a583-f077-47bb-9966-8c8f8d6c4770,0.249563,2025-12-19T13:13:05.812101,2025-12-19T13:13:05.812101,beta_demo_model_sil,v1,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.249563,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2024-07-20 17:03:25,2024-07-20 17:07:22,2024-07
4,1453351,6be3a40f-fd81-4f63-9ef5-881fb740d014,0.163831,2025-12-19T13:12:55.323854,2025-12-19T13:12:55.323854,beta_demo_model_sil,v1,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.163831,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2024-05-04 10:43:34,2024-05-04 10:52:07,2024-05


In [53]:
df2 = dfd.copy()

In [54]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (664741, 20)


In [55]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (664741, 20)
The shape of the dataframe after dropping duplicates is: (664741, 20)


In [56]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [57]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (370, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-01     score     Training   0.000000         321067             v1   
1   2025-03     score      Monthly   0.212490            414             v1   
2   2025-04     score      Monthly   0.271465           1039             v1   
3   2025-05     score      Monthly   0.157399           1561             v1   
4   2025-06     score      Monthly   0.154095          16368             v1   
5   2025-07     score      Monthly   0.187953          22790             v1   
6   2025-08     score      Monthly   0.196011          23270             v1   
7   2025-09     score      Monthly   0.206522          21413             v1   
8   2025-10     score      Monthly   0.280052          24822             v1   

In [58]:
score_output_df

score_output_df['Model_Name'] = 'beta_demo_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'sil_beta_demo_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-01,sil_beta_demo_score,Training,0.000000,321067,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_sil
1,2025-03,sil_beta_demo_score,Monthly,0.212490,414,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_sil
2,2025-04,sil_beta_demo_score,Monthly,0.271465,1039,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_sil
3,2025-05,sil_beta_demo_score,Monthly,0.157399,1561,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_sil
4,2025-06,sil_beta_demo_score,Monthly,0.154095,16368,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
365,2025-11,sil_beta_demo_score,Monthly,17.503315,6,v2,Trench 2,SIL Competitor,Mall,android,beta_demo_model_sil
366,2025-12,sil_beta_demo_score,Monthly,2.937449,55,v2,Trench 2,SIL Competitor,Mall,android,beta_demo_model_sil
367,2025-01,sil_beta_demo_score,Training,0.000000,2,v2,Trench 2,SIL Competitor,Mall,ios,beta_demo_model_sil
368,2025-11,sil_beta_demo_score,Monthly,22.332704,1,v2,Trench 2,SIL Competitor,Mall,ios,beta_demo_model_sil


In [59]:
psi_results = score_output_df.copy()

table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4f26f691-2e31-4128-beff-5d1586c282d3>

## Beta SIL STACK Score Model

### Test

In [60]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - StackScoreModel' then 'beta_stack_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_stack_score
  prediction as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base
where lower(new_loan_type) like '%sil%'
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1129458,313f8879-4013-418a-b0aa-0fce8dbeceae,0.21089442687539978,2025-07-09 10:12:30.565744,2025-07-09 10:12:30.569599,beta_stack_model_sil,v1,SIL Competitor,F,Mall,ios,beta_stack_model_sil,SIL,ALL,Test,0.21089442687539978,"{""s_apps_score"": null, ""s_credo_score"": 0.1146...",2025-07-09 18:12:28,NaT,2025-07
1,1229517,a0989b80-718c-4294-8524-db95fa35c800,0.47629638521846723,2025-12-20 06:39:09.087132,2025-12-20 06:39:09.111480,beta_stack_model_sil,v2,SIL-Instore,F,Appliance,ios,beta_stack_model_sil,SIL,Trench 2,Test,0.47629638521846723,"{""stackScoreModel"": {""sb_demo_score"": 0.548437...",2025-12-20 14:39:06,NaT,2025-12
2,1299613,c2358f9e-7693-4ca6-bfee-9766af3b856a,0.10677885285431735,2025-08-10 02:33:47.351257,2025-08-10 02:33:47.354926,beta_stack_model_sil,v1,SIL ZERO,M,Appliance,ios,beta_stack_model_sil,SIL,ALL,Test,0.10677885285431735,"{""s_apps_score"": null, ""s_credo_score"": 0.089,...",2025-08-10 10:33:44,2025-08-10 10:42:26,2025-08
3,1618714,1f19c957-5e04-4b50-a43d-22b478ebe3c5,0.04734413827332267,2025-12-08 02:38:02.910231,2025-12-08 02:38:02.914920,beta_stack_model_sil,v1,SIL-Instore,M,Appliance,android,beta_stack_model_sil,SIL,ALL,Test,0.04734413827332267,"{""s_apps_score"": 0.44969811960192774, ""s_credo...",2025-12-08 10:37:59,2025-12-08 10:46:32,2025-12
4,1737241,293993a5-b50a-410c-8aad-e219392b8ca0,0.11368764423840695,2025-09-06 12:07:14.785246,2025-09-06 12:07:14.788785,beta_stack_model_sil,v1,SIL-Instore,F,Appliance,ios,beta_stack_model_sil,SIL,ALL,Test,0.11368764423840695,"{""s_apps_score"": null, ""s_credo_score"": 0.0794...",2025-09-06 20:07:12,NaT,2025-09


In [61]:
df1 = dfd.copy()

### Train

In [62]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - StackScoreModel' then 'beta_stack_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  prediction as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
where lower(new_loan_type) like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1023101,095ff5f3-304e-4903-a470-a37656362305,0.062546,2025-12-19T13:14:09.011358,2025-12-19T13:14:09.011358,beta_stack_model_sil,v1,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.062546,"{""apps_score"": 0.3683811988627054, ""credo_gen_...",2023-12-16 19:04:41,2023-12-16 19:10:01,2023-12
1,1071498,42353a87-a544-4493-89a7-899aac930f8f,0.099814,2025-12-19T13:14:28.698977,2025-12-19T13:14:28.698977,beta_stack_model_sil,v1,SIL-Instore,M,Appliance,ios,beta_demo_model_sil,SIL,ALL,Train,0.099814,"{""credo_gen_score"": 0.113026928927845, ""beta_d...",2024-11-25 11:49:22,2024-11-25 12:07:25,2024-11
2,1169961,79851ea7-89a4-4170-bba7-6fcb8c6bdd59,0.812915,2025-12-19T13:23:00.533246,2025-12-19T13:23:00.533246,beta_stack_model_sil,v2,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,Trench 2,Train,0.812915,"{""sb_demo_score"": 0.6203079312242076, ""apps_sc...",2024-12-24 14:09:11,NaT,2024-12
3,1233123,eb516dba-28da-4010-b2f5-dcc853a39152,0.214661,2025-12-19T13:14:10.443312,2025-12-19T13:14:10.443312,beta_stack_model_sil,v1,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.214661,"{""apps_score"": 0.6430911484955417, ""credo_gen_...",2024-07-30 10:57:21,NaT,2024-07
4,1256810,96afcbc7-3fa7-45d3-97f6-67150aa8dc91,0.045134,2025-12-19T13:14:19.185688,2025-12-19T13:14:19.185688,beta_stack_model_sil,v1,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.045134,"{""apps_score"": 0.47581797603005066, ""credo_gen...",2024-02-08 11:13:31,2024-02-08 11:28:12,2024-02


In [63]:
df2 = dfd.copy()

In [64]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (684869, 20)


In [65]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (684869, 20)
The shape of the dataframe after dropping duplicates is: (684869, 20)


In [66]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [67]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (370, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-02     score     Training   0.000000         321067             v1   
1   2025-03     score      Monthly   0.135260            414             v1   
2   2025-04     score      Monthly   0.039789           1039             v1   
3   2025-05     score      Monthly   0.042864           1561             v1   
4   2025-06     score      Monthly   0.049583          16368             v1   
5   2025-07     score      Monthly   0.063482          22790             v1   
6   2025-08     score      Monthly   0.058819          23270             v1   
7   2025-09     score      Monthly   0.055673          21413             v1   
8   2025-10     score      Monthly   0.074304          24822             v1   

In [68]:
score_output_df

score_output_df['Model_Name'] = 'beta_stack_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'sil_beta_stack_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-02,sil_beta_stack_score,Training,0.000000,321067,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_sil
1,2025-03,sil_beta_stack_score,Monthly,0.135260,414,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_sil
2,2025-04,sil_beta_stack_score,Monthly,0.039789,1039,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_sil
3,2025-05,sil_beta_stack_score,Monthly,0.042864,1561,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_sil
4,2025-06,sil_beta_stack_score,Monthly,0.049583,16368,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
365,2025-11,sil_beta_stack_score,Monthly,8.782935,6,v2,Trench 2,SIL Competitor,Mall,android,beta_stack_model_sil
366,2025-12,sil_beta_stack_score,Monthly,4.592781,55,v2,Trench 2,SIL Competitor,Mall,android,beta_stack_model_sil
367,2025-02,sil_beta_stack_score,Training,0.000000,4,v2,Trench 2,SIL Competitor,Mall,ios,beta_stack_model_sil
368,2025-11,sil_beta_stack_score,Monthly,21.639557,1,v2,Trench 2,SIL Competitor,Mall,ios,beta_stack_model_sil


In [69]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=60d37407-d28f-412d-b7bc-6434ac21d58c>

## Cash

### Queries

### Alpha-Cash-CIC-Model (All Trench)

#### Test

In [70]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in('Alpha-Cash-CIC-Model','Alpha Cash CIC Model') then 'cic_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,
 prediction score
 -- aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'cic_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
""" 

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 126c92fa-32a0-4e40-8a1a-aabdf77579b0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1003339,492f01cb-1f67-4fff-9ca5-4de7eeab4630,0.5322520435019061,2025-12-08 08:58:46.168618,2025-12-08 08:58:46.259485,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 1335, ""...",2025-12-08 16:58:39,2025-12-17 17:36:49,2025-12
1,1012701,dcd0fd6f-66f2-4dd9-bb7c-f405518b6736,0.4557388245985053,2025-11-07 06:14:58.534397,2025-11-07 06:14:58.835967,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 8311.0,...",2025-11-07 14:14:36,NaT,2025-11
2,1020527,665d9aa5-a107-48d2-b823-fb711fa711df,0.3451333752361219,2025-11-14 01:43:44.158514,2025-11-14 01:43:44.415796,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 3,Test,"{""cic_max_age_all_contracts_snapshot"": 2532.0,...",2025-11-14 09:43:28,2025-11-14 09:50:50,2025-11
3,1024765,7ccad3d2-f9a9-4077-b2f5-7f5dce1fa793,0.5136344809418468,2025-12-10 03:47:04.886069,2025-12-10 03:47:05.345261,cic_model_cash,v1,Quick,M,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 522.0, ...",2025-12-10 11:46:51,NaT,2025-12
4,1066204,ef64b04b-09e5-4264-86f3-bd1f17651825,0.4725709765498542,2025-11-22 06:58:06.863082,2025-11-22 06:58:06.957790,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 92, ""ci...",2025-11-22 14:57:52,NaT,2025-11


In [71]:
df1 = dfd.copy()

#### Train

In [72]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in('Alpha-Cash-CIC-Model','Alpha Cash CIC Model') then 'cic_model_cash' end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,
 prediction score 
 ,start_time,end_time,
  r.modelDisplayName,
  r.modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'cic_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


Job ID edf79dfc-931d-4cf2-bba8-643f6de80845 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1023913,aff3ae91-3577-4863-a79a-8ed5f317944f,0.626652,2025-12-19T13:18:39.696662,2025-12-19T13:18:39.696662,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 724.0, ""rat...",2024-12-21 21:36:46,NaT,2024-12
1,1027985,127641e5-d8fe-45f6-8e46-ee4e8e990684,0.623976,2025-12-19T13:18:37.657365,2025-12-19T13:18:37.657365,cic_model_cash,v1,Quick,M,not applicable,ios,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 1750.0, ""ra...",2025-01-30 15:13:26,2025-01-30 15:17:37,2025-01
2,1076915,17403810-ff1d-432b-b06f-481075e3dbb3,0.470269,2025-12-19T13:18:39.273390,2025-12-19T13:18:39.273390,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 1154.0, ""ra...",2025-01-23 11:08:26,NaT,2025-01
3,1097239,9e33c764-fb3f-4a4b-9e3f-77b641b6bb31,0.463703,2025-12-19T13:18:38.493336,2025-12-19T13:18:38.493336,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 934.0, ""rat...",2024-09-19 08:07:33,2024-09-20 09:34:44,2024-09
4,1100187,0fb321f3-5597-414d-a80b-f16631725283,0.380968,2025-12-19T13:18:38.706609,2025-12-19T13:18:38.706609,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 3528.0, ""ra...",2024-10-29 21:39:26,2024-10-29 21:42:37,2024-10


In [73]:
df2 = dfd.copy()

In [74]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (114301, 19)


In [75]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (114301, 19)
The shape of the dataframe after dropping duplicates is: (114301, 19)


In [76]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [77]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
2138,1935855,702d712d-6bc0-4078-99d1-a432e5f156b1,0.658680,2025-12-14 04:33:50.542628,2025-12-14 04:33:50.644803,cic_model_cash,v1,Quick,M,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 514, ""c...",2025-12-14 12:33:22,NaT,2025-12
96498,2808671,af2868fc-7e8f-4d96-940b-e81e114d7c17,0.535448,2025-12-19T13:18:24.187461,2025-12-19T13:18:24.187461,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 1,Train,"{""max_age_all_contracts_snapshot"": 152.0, ""rat...",2024-09-01 07:10:39,NaT,2024-09
67131,3196012,5dfb72c1-d9bf-4cd4-9dda-316e9130cee9,0.466055,2025-12-19T13:18:25.667643,2025-12-19T13:18:25.667643,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 1,Train,"{""max_age_all_contracts_snapshot"": 592.0, ""rat...",2025-01-17 13:57:28,2025-01-18 10:12:28,2025-01
72926,2670986,70f19a76-1127-4aa0-a613-d4b43be4369e,0.415348,2025-12-19T13:18:39.034950,2025-12-19T13:18:39.034950,cic_model_cash,v1,Quick,M,not applicable,ios,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 1445.0, ""ra...",2024-11-02 10:49:03,2024-11-02 11:02:20,2024-11
26668,3788860,72493ffb-4c05-4045-a725-cff6b6a93bd7,0.536174,2025-11-03 05:31:13.627832,2025-11-03 05:31:13.763954,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 1,Test,"{""cic_max_age_all_contracts_snapshot"": 365, ""c...",2025-11-03 13:31:02,NaT,2025-11
55408,2677879,c28eba0c-d85a-4987-a7f6-cd19a89782eb,0.455744,2025-12-19T13:18:38.627751,2025-12-19T13:18:38.627751,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 189.0, ""rat...",2024-11-17 15:07:00,2024-11-18 13:43:37,2024-11
18690,2968389,376b58db-9e76-498b-9757-bb87ba0a60d2,0.765294,2025-12-13 19:23:11.672660,2025-12-13 19:23:11.778872,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 162, ""c...",2025-12-14 03:23:00,NaT,2025-12
3748,2228865,225bb089-a978-45cc-a2cc-986c3935a1a6,0.508149,2025-12-22 10:54:43.474021,2025-12-22 10:54:43.651676,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 1887.0,...",2025-12-22 18:54:26,NaT,2025-12
3489,3746340,5e987162-bfb1-479c-b10a-e5ae7fc4845a,0.584014,2025-10-15 12:19:22.885598,2025-10-15 12:19:23.025092,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 1,Test,"{""cic_max_age_all_contracts_snapshot"": 731, ""c...",2025-10-15 20:19:13,NaT,2025-10
76663,2832231,d08071e4-4a42-40c4-8b5b-e3c24e04b0c4,0.757970,2025-12-19T13:18:28.949095,2025-12-19T13:18:28.949095,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 1,Train,"{""max_age_all_contracts_snapshot"": 182.0, ""rat...",2024-09-09 08:09:26,NaT,2024-09


### PSI calculation

In [78]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (80, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-01     score     Training   0.000000          62242             v1   
1   2025-09     score      Monthly   0.173635            134             v1   
2   2025-10     score      Monthly   0.022447           9210             v1   
3   2025-11     score      Monthly   0.026010          18803             v1   
4   2025-12     score      Monthly   0.085000          23912             v1   
5   2025-01     score     Training   0.000000          18610             v1   
6   2025-09     score      Monthly   2.455029             31             v1   
7   2025-10     score      Monthly   0.052349           2355             v1   
8   2025-11     score      Monthly   0.023737          10225             v1   


In [79]:
score_output_df
score_output_df['Model_Name'] = 'cic_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'cic_model_cash_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-01,cic_model_cash_score,Training,0.000000,62242,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
1,2025-09,cic_model_cash_score,Monthly,0.173635,134,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
2,2025-10,cic_model_cash_score,Monthly,0.022447,9210,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
3,2025-11,cic_model_cash_score,Monthly,0.026010,18803,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
4,2025-12,cic_model_cash_score,Monthly,0.085000,23912,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
5,2025-01,cic_model_cash_score,Training,0.000000,18610,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
6,2025-09,cic_model_cash_score,Monthly,2.455029,31,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
7,2025-10,cic_model_cash_score,Monthly,0.052349,2355,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
8,2025-11,cic_model_cash_score,Monthly,0.023737,10225,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,cic_model_cash
9,2025-12,cic_model_cash_score,Monthly,0.087232,13997,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,cic_model_cash


In [80]:
psi_results = score_output_df.copy()
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dc02e7f5-f141-47f3-82b9-60201d2e5751>

### Alpha-Cash-Stack-Model

#### Test

In [81]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when  modelDisplayName in ('Alpha-Cash-Stack-Model') then 'alpha_stack_model_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'alpha_stack_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 30874557-4eab-4cc4-9b44-92ad598cf86c successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1008840,8a0de6b2-75f3-4605-a407-2f8c439c71aa,2025-10-30 20:43:29.420201,2025-10-30 20:43:29.580532,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.6151913818434823, ""c_demo_sco...",0.6003628763737517,2025-10-31 04:43:18,NaT,2025-10
1,1026845,4d964431-54c8-47b0-a4b5-7ff4f0d39a58,2025-11-12 13:51:15.635586,2025-11-12 13:51:15.784658,alpha_stack_model_cash,v1,Quick,M,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": null, ""c_demo_score"": 0.3757157...",0.188253926421539,2025-11-12 21:51:01,NaT,2025-11
2,1092988,09c7cd46-ea49-4551-bf2a-d61304235170,2025-11-13 07:55:19.065767,2025-11-13 07:55:19.551451,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.35770405610793143, ""c_demo_sc...",0.26054108842446916,2025-11-13 15:55:07,2025-11-13 15:58:20,2025-11
3,1098698,bc41551a-2dbc-4bc1-8ab5-72eba9562677,2025-12-13 04:15:51.957475,2025-12-13 04:15:52.262420,alpha_stack_model_cash,v1.1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.5016462875259132, ""c_demo_sco...",0.4970194489405544,2025-12-13 12:15:32,NaT,2025-12
4,1183159,b2feebb8-1184-4058-b029-3e5ecab96c53,2025-10-14 08:00:26.008076,2025-10-14 08:00:26.149857,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.46586302342005004, ""c_demo_sc...",0.33653658321804303,2025-10-14 16:00:16,2025-10-14 16:16:36,2025-10


In [82]:
df1 = dfd.copy()

#### Train

In [83]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when  modelDisplayName in ('Alpha-Cash-Stack-Model') then 'alpha_stack_model_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction score 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'alpha_stack_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'

"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


Job ID aa56081a-f7c7-48a3-929d-008051a93637 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1052618,a0a121b9-2c2e-4d6a-bb88-28daa2106df7,0.220547,2025-12-19T13:18:16.168138,2025-12-19T13:18:16.168138,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 3,Train,"{""demo_score"": 0.294558839965906, ""apps_score""...",2024-11-12 18:28:37,2024-11-12 18:42:18,2024-11
1,1061097,97bb055a-bb83-4ea7-8178-45cdb32a9bca,0.031670,2025-12-19T13:18:05.872177,2025-12-19T13:18:05.872177,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.4381838688516576, ""apps_score...",2024-11-04 01:37:37,2024-11-04 12:51:29,2024-11
2,1064116,9857828f-7196-463d-848a-613370a67ae7,0.300733,2025-12-19T13:18:05.473521,2025-12-19T13:18:05.473521,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.4493886982049852, ""apps_score...",2025-01-13 14:49:45,2025-01-25 18:30:26,2025-01
3,1079396,c08b1117-c8fa-474c-91e9-7845ec93559d,0.472758,2025-12-19T13:18:06.951205,2025-12-19T13:18:06.951205,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5015331196630954, ""credo_scor...",2024-12-06 12:21:05,NaT,2024-12
4,1101840,c3976d06-4474-4e4c-8ee4-7fb86d2da802,0.333932,2025-12-19T13:18:05.860174,2025-12-19T13:18:05.860174,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.4141833632592463, ""apps_score...",2024-10-06 08:10:02,NaT,2024-10


In [84]:
df2 = dfd.copy()

In [85]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (108859, 19)


In [86]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (108859, 19)
The shape of the dataframe after dropping duplicates is: (108859, 19)


In [87]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [88]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
3087,2805675,ac51ea25-7e45-4d93-bd4c-306a8648f499,2025-11-04 06:06:22.646694,2025-11-04 06:06:22.951735,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.49760920816135745, ""c_demo_sc...",0.509737,2025-11-04 14:05:58,2025-11-04 14:32:59,2025-11
17702,3289012,c913e5e4-faa7-4972-aeda-1f7f6f757c1e,2025-12-21 04:27:23.744708,2025-12-21 04:27:23.963440,alpha_stack_model_cash,v1.1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.5132942442012038, ""c_demo_sco...",0.647960,2025-12-21 12:26:46,NaT,2025-12
10730,3789083,2f16f6f2-2ea3-4142-8945-518172709037,2025-11-03 06:38:55.512846,2025-11-03 06:38:55.634585,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": null, ""c_demo_score"": 0.3046351...",0.219273,2025-11-03 14:38:45,NaT,2025-11
25750,3868578,c393f93b-1c75-4d9b-ac72-dde41437925c,2025-12-07 03:48:13.727284,2025-12-07 03:48:13.888159,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": 0.5247131745175215, ""c_demo_sco...",0.641913,2025-12-07 11:47:40,NaT,2025-12
102921,3097624,8c2c94b6-01bf-43b0-b17f-b3cb305fa72c,2025-12-19T13:17:54.374180,2025-12-19T13:17:54.374180,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5028952745468762, ""apps_score...",0.792429,2024-12-13 07:29:54,NaT,2024-12
50498,3869323,2fbf99b6-bc4b-4db0-bb75-3113289d3187,2025-12-07 06:11:57.017480,2025-12-07 06:11:57.211960,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": 0.5571805116331441, ""c_demo_sco...",0.677227,2025-12-07 14:11:31,NaT,2025-12
84370,3165316,513df168-5a3d-4a0d-a038-14e0f2844c01,2025-12-19T13:17:56.935283,2025-12-19T13:17:56.935283,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.45037255481294464, ""credo_sco...",0.338931,2025-01-08 20:10:08,NaT,2025-01
104772,1794761,b0b8c565-5701-4273-b688-a323d96286c4,2025-12-19T13:18:16.133014,2025-12-19T13:18:16.133014,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 3,Train,"{""demo_score"": 0.5296591459878609, ""apps_score...",0.658205,2024-12-19 19:48:31,2024-12-19 20:06:50,2024-12
51809,3676626,9b2f928f-7895-4874-9ceb-2c79362201d0,2025-12-20 01:03:45.086004,2025-12-20 01:03:45.204645,alpha_stack_model_cash,v1.1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.4964186967060142, ""c_demo_sco...",0.503181,2025-12-20 09:03:30,NaT,2025-12
45846,2468647,a067a994-6fd0-4aae-80f0-c81317c22c83,2025-12-17 11:46:58.652560,2025-12-17 11:46:58.745340,alpha_stack_model_cash,v1.1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.5163912388534552, ""c_demo_sco...",0.616939,2025-12-17 19:46:46,NaT,2025-12


### PSI calculation

In [89]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (80, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-02     score     Training   0.000000          56800             v1   
1   2025-09     score      Monthly   2.367048            134             v1   
2   2025-10     score      Monthly   0.158018           9210             v1   
3   2025-11     score      Monthly   0.158527          18803             v1   
4   2025-12     score      Monthly   0.162296          11534             v1   
5   2025-02     score     Training   0.000000          17248             v1   
6   2025-09     score      Monthly   4.359549             31             v1   
7   2025-10     score      Monthly   0.450848           2355             v1   
8   2025-11     score      Monthly   0.357696          10225             v1   


In [90]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'alpha_stack_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'alpha_stack_model_cash_score')
score_output_df

The shape of the score_output_df is (80, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-02,alpha_stack_model_cash_score,Training,0.000000,56800,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
1,2025-09,alpha_stack_model_cash_score,Monthly,2.367048,134,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
2,2025-10,alpha_stack_model_cash_score,Monthly,0.158018,9210,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
3,2025-11,alpha_stack_model_cash_score,Monthly,0.158527,18803,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
4,2025-12,alpha_stack_model_cash_score,Monthly,0.162296,11534,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
5,2025-02,alpha_stack_model_cash_score,Training,0.000000,17248,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
6,2025-09,alpha_stack_model_cash_score,Monthly,4.359549,31,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
7,2025-10,alpha_stack_model_cash_score,Monthly,0.450848,2355,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
8,2025-11,alpha_stack_model_cash_score,Monthly,0.357696,10225,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash
9,2025-12,alpha_stack_model_cash_score,Monthly,0.391588,5907,v1,Trench 2,OVER ALL,OVER ALL,OVER ALL,alpha_stack_model_cash


In [91]:
psi_results = score_output_df.copy()
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=315393af-b8cf-4c0e-b12c-2c1f7437f510>

### Beta-Cash-Demo-Model

#### Test

In [92]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Demo-Model') then 'beta_demo_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId
 ,start_time,end_time,  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_demo_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 9b3a7ea0-52c2-43c8-a98d-027ecefa62fc successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1021052,f79fc5e6-0f42-4688-b2ed-53905aea0c42,2025-11-18 16:08:21.882434,2025-11-18 16:08:22.452476,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.5789911596353201,2025-11-19 00:08:18,NaT,2025-11
1,1030544,22c99c1d-f037-4cbe-b47f-5d89fbadec1e,2025-11-15 21:26:27.387239,2025-11-15 21:26:27.883346,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.5350259708089844,2025-11-16 05:26:24,2025-11-16 10:32:36,2025-11
2,1040558,d57a3939-2322-437c-bcf9-b106d3dba6db,2025-11-15 19:24:01.610977,2025-11-15 19:24:02.235670,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.4955941110615301,2025-11-16 03:23:58,NaT,2025-11
3,1075493,fe8a44a1-00f5-42ad-81de-d31aa8cb69c9,2025-11-12 12:28:34.912978,2025-11-12 12:28:35.701572,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.44208954995282873,2025-11-12 20:28:32,2025-11-13 19:53:20,2025-11
4,1096716,c47a6cfc-4b26-4908-ac68-269e2a83d6d1,2025-11-15 01:25:00.860369,2025-11-15 01:25:01.293829,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.6003053759521715,2025-11-15 09:24:58,NaT,2025-11


In [93]:
df1 = dfd.copy()

#### Train

In [94]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Demo-Model') then 'beta_demo_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_demo_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


Job ID 0be398a4-7e9e-4a40-aa81-f13b4e939c90 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1003545,b0ff667e-b78f-437d-b8da-4a0ec151076b,0.411368,2025-12-19T13:19:31.863992,2025-12-19T13:19:31.863992,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.411368,2025-04-22 20:36:52,NaT,2025-04
1,1003793,f67896c3-2585-4631-8867-393ea1d9f236,0.538331,2025-12-19T13:19:37.428179,2025-12-19T13:19:37.428179,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.538331,2025-01-21 13:09:01,NaT,2025-01
2,1009978,b92ef3b7-28cd-490e-a0a8-96caba93c80a,0.550315,2025-12-19T13:19:37.613872,2025-12-19T13:19:37.613872,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.550315,2025-08-18 23:00:11,NaT,2025-08
3,1013631,706d87a9-549b-4549-8920-23aeccf4add7,0.476642,2025-12-19T13:19:29.774900,2025-12-19T13:19:29.774900,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.476642,2025-05-28 19:05:01,2025-05-28 22:35:09,2025-05
4,1014362,8dd088fc-9c8c-4419-801f-28e7e944d0c1,0.482876,2025-12-19T13:19:34.453724,2025-12-19T13:19:34.453724,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.482876,2024-10-23 16:50:54,NaT,2024-10


In [95]:
df2 = dfd.copy()

In [96]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (434699, 20)


In [97]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (434699, 20)
The shape of the dataframe after dropping duplicates is: (434699, 20)


In [98]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [99]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month,prediction
333746,1962213,b394a9d1-a0ac-4a38-8fa3-c19194c5c00b,2025-12-19T13:19:25.034769,2025-12-19T13:19:25.034769,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.574391,2025-05-22 12:19:53,NaT,2025-05,0.574391
95868,2545487,b3c3ae99-c3e1-4702-bde0-0e0b82f24ce1,2025-11-28 09:51:13.576921,2025-11-28 09:51:14.031411,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.483029,2025-11-28 17:51:10,2025-11-28 17:55:18,2025-11,NaN
130422,1211431,b3274916-3dda-4a90-8ec6-315e7309e9f6,2025-10-30 05:32:17.261978,2025-10-30 05:32:18.177477,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.504402,2025-10-30 13:32:14,NaT,2025-10,NaN
111949,3732758,925af67c-dc27-4e6b-bd6f-a29d749fa5b0,2025-10-09 01:58:22.287037,2025-10-09 01:58:22.546566,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.599236,2025-10-09 09:58:20,NaT,2025-10,NaN
7871,3749570,fa5cbe72-930b-445c-817c-c44a92aa56e8,2025-10-27 21:24:12.515016,2025-10-27 21:24:12.762199,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.488943,2025-10-28 05:24:09,NaT,2025-10,NaN
415824,3073664,73cf4937-98ef-463a-a34e-393702169c24,2025-12-19T13:19:11.806838,2025-12-19T13:19:11.806838,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.485480,2024-12-02 20:12:29,NaT,2024-12,0.485480
255567,2200019,c6c515c2-5869-4401-9e9d-a7dc4562a51b,2025-12-19T13:19:34.668393,2025-12-19T13:19:34.668393,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.403328,2025-04-14 15:08:34,NaT,2025-04,0.403328
51700,3743389,9df1acf3-9a3d-42d0-bfa8-7d13416fb330,2025-10-14 03:45:36.417998,2025-10-14 03:45:36.433329,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.483283,2025-10-14 11:45:34,2025-10-14 11:56:07,2025-10,NaN
352879,2955969,229c6fb6-ef95-43da-895c-65695747a205,2025-12-19T13:19:04.600646,2025-12-19T13:19:04.600646,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.660221,2024-10-19 22:40:22,NaT,2024-10,0.660221
195747,2691758,81d41de6-81b2-4731-8551-ecd63fdb6391,2025-12-19T13:19:21.958587,2025-12-19T13:19:21.958587,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.570133,2024-10-30 08:53:10,NaT,2024-10,0.570133


### PSI calculation

In [100]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (80, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-09     score     Training   0.000000         260796             v1   
1   2025-09     score      Monthly   0.019354            803             v1   
2   2025-10     score      Monthly   0.002818          35272             v1   
3   2025-11     score      Monthly   0.019509          58553             v1   
4   2025-12     score      Monthly   0.022484          79275             v1   
5   2025-09     score     Training   0.000000         125668             v1   
6   2025-09     score      Monthly   0.030544            578             v1   
7   2025-10     score      Monthly   0.005559          24909             v1   
8   2025-11     score      Monthly   0.014814          26229             v1   


In [101]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'beta_demo_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'beta_demo_model_cash_score')
score_output_df

The shape of the score_output_df is (80, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-09,beta_demo_model_cash_score,Training,0.000000,260796,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
1,2025-09,beta_demo_model_cash_score,Monthly,0.019354,803,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
2,2025-10,beta_demo_model_cash_score,Monthly,0.002818,35272,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
3,2025-11,beta_demo_model_cash_score,Monthly,0.019509,58553,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
4,2025-12,beta_demo_model_cash_score,Monthly,0.022484,79275,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
5,2025-09,beta_demo_model_cash_score,Training,0.000000,125668,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
6,2025-09,beta_demo_model_cash_score,Monthly,0.030544,578,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
7,2025-10,beta_demo_model_cash_score,Monthly,0.005559,24909,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
8,2025-11,beta_demo_model_cash_score,Monthly,0.014814,26229,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash
9,2025-12,beta_demo_model_cash_score,Monthly,0.040441,30330,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_demo_model_cash


In [102]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result() 

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2bca5727-1d68-46fe-9c3d-d5d153d5f2a8>

### Beta-Cash-AppScore-Model

#### Test

In [103]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-AppScore-Model') then 'apps_score_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,start_time,end_time,
prediction,
case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'apps_score_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 64bce906-be6d-4eb8-8980-00e4df68fcc9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1000979,0f1e9641-e8f4-427f-8709-157e5b81295a,2025-11-18 10:57:22.032828,2025-11-18 10:57:22.735555,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",0.698837,2025-11-18 18:57:18,NaT,2025-11
1,1003674,c6f4a07d-9104-446a-beed-2dbbbe5426fe,2025-11-05 02:25:40.728182,2025-11-05 02:25:41.569220,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 6, ""app_cn...",0.241139,2025-11-05 10:25:32,NaT,2025-11
2,1020725,cc32dbc9-6c2a-437a-9149-e083bfb46b98,2025-11-14 02:42:00.894418,2025-11-14 02:42:01.681716,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 3, ""app_cn...",0.376522,2025-11-14 10:41:57,2025-11-14 14:18:39,2025-11
3,1034017,d46bce78-7d77-490f-9e18-7d75f9b9c348,2025-11-15 19:54:51.058753,2025-11-15 19:54:51.978911,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 5, ""app_cn...",0.441451,2025-11-16 03:54:47,2025-11-16 14:10:05,2025-11
4,1058717,fdacbe20-35e0-46d1-b265-94454560eee4,2025-12-17 06:41:15.558652,2025-12-17 06:41:16.747170,apps_score_cash,v1.1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_rated_for_18plus_ever"": 0, ""app_cnt_...",0.512111,2025-12-17 14:41:13,NaT,2025-12


In [104]:
df1 = dfd.copy()

#### Train

In [105]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in  ('Beta-Cash-AppScore-Model') then 'apps_score_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,start_time,end_time,prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'apps_score_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 8bbd69a6-a618-4986-af46-a0850a14897c successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1001162,c43b1568-d856-4630-b94f-639a4472c485,2025-12-19T13:21:17.558284,2025-12-19T13:21:17.558284,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.4683486924242265, ""a...",0.468349,2025-01-21 23:58:46,2025-01-23 14:40:20,2025-01
1,1021774,f09d0438-214c-453b-b9da-e021c92797d5,2025-12-19T13:21:17.371606,2025-12-19T13:21:17.371606,apps_score_cash,v1,ACL TSA,F,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.509331631840936, ""ap...",0.509332,2024-12-13 20:28:31,NaT,2024-12
2,1033558,4beaaa63-b2e5-4aa6-8487-35e04423ffa4,2025-12-19T13:21:15.688496,2025-12-19T13:21:15.688496,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.6397305925561246, ""a...",0.639731,2024-11-10 08:06:55,NaT,2024-11
3,1049379,86ca2834-b098-4dc9-8899-9f58fc9c1eb4,2025-12-19T13:21:17.110203,2025-12-19T13:21:17.110203,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.5380068534224882, ""a...",0.538007,2024-09-23 17:25:09,NaT,2024-09
4,1067982,973f4d06-a960-437e-ae7f-b95c52fa1e86,2025-12-19T13:21:16.650185,2025-12-19T13:21:16.650185,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.6014506242894095, ""a...",0.601451,2024-10-07 09:47:20,NaT,2024-10


In [106]:
df2 = dfd.copy()

In [107]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (309390, 19)


In [108]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (309390, 19)
The shape of the dataframe after dropping duplicates is: (309390, 19)


In [109]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [110]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
274212,3143903,9a8ca520-7c1d-46b0-98b4-5d94133e87ec,2025-12-19T13:20:53.580633,2025-12-19T13:20:53.580633,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.7015789740375017, ""a...",0.701579,2024-12-27 11:13:36,NaT,2024-12
98939,2925991,aeafd0aa-86c0-4352-984f-b9155c2273dc,2025-12-10 02:03:05.746585,2025-12-10 02:03:06.368169,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",0.513668,2025-12-10 10:03:01,NaT,2025-12
95480,3668862,295f0195-cb16-43a9-bec8-d1c0f73a3392,2025-12-07 11:40:10.286077,2025-12-07 11:40:11.329635,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",0.567356,2025-12-07 19:40:07,NaT,2025-12
298000,3072461,75928296-c63a-45e8-b56d-4e4c67139331,2025-12-19T13:20:43.036991,2025-12-19T13:20:43.036991,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.4989068129074718, ""a...",0.498907,2024-12-02 14:23:33,2024-12-02 19:14:41,2024-12
9010,3068556,cc8a0a2d-4df6-4a7e-a808-937aa0488232,2025-12-03 13:02:27.185081,2025-12-03 13:02:27.906417,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",0.421122,2025-12-03 21:02:24,NaT,2025-12
227812,3146118,cf6bc00e-620f-4171-bb78-ff24e10c212d,2025-12-19T13:20:51.178602,2025-12-19T13:20:51.178602,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.5325862393167745, ""a...",0.532586,2024-12-27 19:34:23,NaT,2024-12
113739,3886617,c35aae85-f24d-4888-b617-e8627b3e8940,2025-12-11 16:47:34.552058,2025-12-11 16:47:34.822989,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Test,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",0.586214,2025-12-12 00:47:31,NaT,2025-12
38322,2493225,751ea9de-db15-456d-b721-0ec95fb26175,2025-12-05 05:39:03.827977,2025-12-05 05:39:04.596665,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",0.537199,2025-12-05 13:39:00,NaT,2025-12
204076,3192761,c008b651-0130-4814-9fc9-eb32b6217cab,2025-12-19T13:20:43.156885,2025-12-19T13:20:43.156885,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.6620161353347216, ""a...",0.662016,2025-01-16 09:44:36,NaT,2025-01
134510,3041861,e229a4e8-8c8d-47fe-8112-22fe493c6e15,2025-12-19T13:20:56.227426,2025-12-19T13:20:56.227426,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.6070545765625206, ""a...",0.607055,2024-11-20 20:55:59,NaT,2024-11


### PSI calculation

In [111]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (68, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-01     score     Training   0.000000         185521             v1   
1   2025-09     score      Monthly   0.059202            521             v1   
2   2025-10     score      Monthly   0.035158          24667             v1   
3   2025-11     score      Monthly   0.054082          40688             v1   
4   2025-12     score      Monthly   0.064188          28023             v1   
5   2025-01     score     Training   0.000000         140968             v1   
6   2025-09     score      Monthly   0.093396            378             v1   
7   2025-10     score      Monthly   0.031453          17704             v1   
8   2025-11     score      Monthly   0.038366          19216             v1   


In [112]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'apps_score_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'apps_score_cash_score')
score_output_df

The shape of the score_output_df is (68, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-01,apps_score_cash_score,Training,0.000000,185521,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
1,2025-09,apps_score_cash_score,Monthly,0.059202,521,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
2,2025-10,apps_score_cash_score,Monthly,0.035158,24667,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
3,2025-11,apps_score_cash_score,Monthly,0.054082,40688,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
4,2025-12,apps_score_cash_score,Monthly,0.064188,28023,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
5,2025-01,apps_score_cash_score,Training,0.000000,140968,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
6,2025-09,apps_score_cash_score,Monthly,0.093396,378,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
7,2025-10,apps_score_cash_score,Monthly,0.031453,17704,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
8,2025-11,apps_score_cash_score,Monthly,0.038366,19216,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,apps_score_cash
9,2025-12,apps_score_cash_score,Monthly,0.040613,13115,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,apps_score_cash


In [113]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result() 

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=22b747f7-5828-4379-ba82-f34c25a0b338>

### Beta-Cash-Stack-Model

#### Test

In [114]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Stack-Model') then 'beta_stack_model_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_stack_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 48eaae0b-0809-4098-b408-e93ebe47291f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1006766,af1ddbed-a118-403a-9053-8aee3f7e1c20,0.4937401135431306,2025-11-24 21:52:52.385087,2025-11-24 21:52:53.181570,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.5490518061543037"", ""credo_sc...",2025-11-25 05:52:49,NaT,2025-11
1,1009434,cd6aa5fe-0de9-4ee6-8c65-a0c2c9aed770,0.32583741934656935,2025-11-12 05:09:45.905190,2025-11-12 05:09:46.623416,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 3,Test,"{""demo_score"": ""0.31525374918787186"", ""credo_s...",2025-11-12 13:09:42,2025-11-12 13:13:01,2025-11
2,1016090,ebac03e8-2902-4c80-8245-08c684a6153c,0.5527520236252742,2025-12-15 20:30:55.615023,2025-12-15 20:30:56.094544,beta_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.5393521519080312"", ""credo_sc...",2025-12-16 04:30:51,2025-12-16 11:06:50,2025-12
3,1019895,f6782b8b-f479-488b-8997-8923c240a80e,0.7221381377126901,2025-11-01 08:56:19.330242,2025-11-01 08:56:19.911768,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.6718011005357388"", ""credo_sc...",2025-11-01 16:56:17,NaT,2025-11
4,1052523,15962638-108b-47d4-97e6-db4682142d1f,0.39266497214987217,2025-12-16 21:25:29.084751,2025-12-16 21:25:29.564739,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 3,Test,"{""demo_score"": ""0.4286915278777596"", ""credo_sc...",2025-12-17 05:25:25,NaT,2025-12


In [115]:
df1 = dfd.copy()

#### Train

In [116]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Stack-Model') then 'beta_stack_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,start_time,end_time,prediction,
case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_stack_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 55131d83-bca3-4b5d-8fa8-b3bda718adcf successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1017244,78511f56-133b-4f9d-93c5-ae70b288e761,0.672538,2025-12-19T13:20:24.254312,2025-12-19T13:20:24.254312,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.6212757744719515, ""apps_score...",2024-11-28 03:18:36,NaT,2024-11
1,1020356,1cd8f8fc-c964-424a-b267-1a0a9825b1f0,0.171166,2025-12-19T13:20:24.135341,2025-12-19T13:20:24.135341,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.459455607462826, ""apps_score""...",2025-01-27 14:59:08,NaT,2025-01
2,1021502,a04c23e0-ee88-4798-887b-09cc7cc3e862,0.601688,2025-12-19T13:20:27.016229,2025-12-19T13:20:27.016229,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.603569145901294, ""credo_score...",2024-12-18 16:14:39,2024-12-18 16:27:07,2024-12
3,1034173,883507a2-b985-4781-80c1-28e0eb425f1e,0.675176,2025-12-19T13:20:24.176406,2025-12-19T13:20:24.176406,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5314770748579316, ""apps_score...",2024-11-30 02:40:04,NaT,2024-11
4,1036180,624aefbc-a3cf-4dd8-8b97-7911273b782e,0.632188,2025-12-19T13:20:24.260312,2025-12-19T13:20:24.260312,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5847907909903063, ""apps_score...",2024-11-13 15:59:13,NaT,2024-11


In [117]:
df2 = dfd.copy()

In [118]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (389021, 19)


In [119]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (389021, 19)
The shape of the dataframe after dropping duplicates is: (389021, 19)


In [120]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [121]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
278727,3105528,32f97642-2f55-4457-a05c-80503e64d4c0,0.922093,2025-12-19T13:20:05.985858,2025-12-19T13:20:05.985858,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5713518515150636, ""apps_score...",2024-12-15 05:48:43,NaT,2024-12
233119,3062789,74d9a82d-08bb-4e0d-9851-1fd5ca863b58,0.645754,2025-12-19T13:20:10.064033,2025-12-19T13:20:10.064033,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5111595609184835, ""credo_scor...",2024-11-29 23:09:04,NaT,2024-11
381645,3073644,f08bad6b-f168-48e9-aec8-99aa8dbc02cc,0.921282,2025-12-19T13:20:02.376350,2025-12-19T13:20:02.376350,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5593831944340839, ""apps_score...",2024-12-02 20:02:20,NaT,2024-12
304752,3239577,bb14dd2a-fcb3-4745-81e3-22943463d741,0.205431,2025-12-19T13:20:02.953894,2025-12-19T13:20:02.953894,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.24634540561830393, ""apps_scor...",2025-02-05 17:11:38,2025-02-06 17:31:41,2025-02
309128,3258659,4a0c43f1-7988-4fe2-a197-cac9c6f501eb,0.926690,2025-12-19T13:20:02.684540,2025-12-19T13:20:02.684540,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5901591176764953, ""apps_score...",2025-02-14 11:00:03,NaT,2025-02
174681,2949135,4ee7678b-fdd7-486b-96f5-9047067c556e,0.645846,2025-12-19T13:20:06.907216,2025-12-19T13:20:06.907216,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5819204047015532, ""apps_score...",2024-10-17 17:14:33,NaT,2024-10
165070,3884298,88684597-5a3c-4a57-816a-251c997b88db,0.780299,2025-12-11 09:35:23.790435,2025-12-11 09:35:24.042425,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Test,"{""demo_score"": ""0.48842697103223176"", ""credo_s...",2025-12-11 17:35:18,NaT,2025-12
128213,3756538,f59c5dc7-2ca6-487b-9514-1d001d0b688c,0.356870,2025-10-20 06:13:33.415274,2025-10-20 06:13:33.691101,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Test,"{""demo_score"": ""0.47096502670070545"", ""credo_s...",2025-10-20 14:13:31,NaT,2025-10
282913,3076554,ec3c4e10-430b-4c69-9d5f-c3edb7a8f616,0.682439,2025-12-19T13:20:09.410161,2025-12-19T13:20:09.410161,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.6155649578961103, ""credo_scor...",2024-12-03 21:06:58,NaT,2024-12
83042,3789801,5569f65b-b819-4d7f-b79d-bc69a397c9c4,0.528615,2025-11-03 11:06:39.701233,2025-11-03 11:06:39.713763,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 1,Test,"{""demo_score"": ""0.5272720908266292"", ""credo_sc...",2025-11-03 19:06:37,NaT,2025-11


### PSI calculation

In [122]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (80, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-02     score     Training   0.000000         215117             v1   
1   2025-09     score      Monthly   0.047982            803             v1   
2   2025-10     score      Monthly   0.013397          35272             v1   
3   2025-11     score      Monthly   0.025004          58553             v1   
4   2025-12     score      Monthly   0.022007          38269             v1   
5   2025-02     score     Training   0.000000         154382             v1   
6   2025-09     score      Monthly   0.048656            578             v1   
7   2025-10     score      Monthly   0.012669          24909             v1   
8   2025-11     score      Monthly   0.014009          26229             v1   


In [123]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'beta_stack_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'beta_stack_model_cash_score')
score_output_df

The shape of the score_output_df is (80, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-02,beta_stack_model_cash_score,Training,0.000000,215117,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
1,2025-09,beta_stack_model_cash_score,Monthly,0.047982,803,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
2,2025-10,beta_stack_model_cash_score,Monthly,0.013397,35272,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
3,2025-11,beta_stack_model_cash_score,Monthly,0.025004,58553,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
4,2025-12,beta_stack_model_cash_score,Monthly,0.022007,38269,v1,OVER ALL,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
5,2025-02,beta_stack_model_cash_score,Training,0.000000,154382,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
6,2025-09,beta_stack_model_cash_score,Monthly,0.048656,578,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
7,2025-10,beta_stack_model_cash_score,Monthly,0.012669,24909,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
8,2025-11,beta_stack_model_cash_score,Monthly,0.014009,26229,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash
9,2025-12,beta_stack_model_cash_score,Monthly,0.011182,16024,v1,Trench 1,OVER ALL,OVER ALL,OVER ALL,beta_stack_model_cash


In [124]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result() 

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dd39d934-50ed-428d-bf34-d60516f3f116>

# End